<div class="alert alert-block alert-warning">
Only Python 3 compatible! Python 2 support has been dropped since v3.0 of this UseCase.
</div>

# Basal Ganglia Single Cell Validation Use Case

***
**Aim: ** To validate single cell models of the Basal Ganglia against data from literature.

***
**Version:** 3.1 (14/01/2020)
***
**Contributors:**  Shailesh Appukuttan (CNRS)
***
**Contact:** [shailesh.appukuttan@unic.cnrs-gif.fr](mailto:shailesh.appukuttan@unic.cnrs-gif.fr)

<div class="alert alert-block alert-success" style="color:black">
<h2>About This Use Case</h2><br />
This test shall take as input a BluePyOpt optimized output file, containing a `hall_of_fame.json` file specifying a collection of parameter sets. The validation test would then evaluate the model for all (or specified) parameter sets against various eFEL features. The results are registered on the HBP Validation Framework app. If an instance of the Model Catalog and Validation Framework are not found in the current Collab, then these are created. Additionally, a test report is generated and this can be viewed within the Jupyter notebook, or downloaded.
</div>

## 1. Seting up the environment

In [ ]:
#Loading the proper version of NEURON
!ln -sfn /home/jovyan/.local/nrn-7.6/ /home/jovyan/.local/nrn

In [ ]:
import os
import pkg_resources
from pkg_resources import parse_version
!pip install -q tornado==4.5.3
!pip install -q --upgrade hbp-service-client

<div class="alert alert-block alert-danger">
<strong>Note:</strong> If you encounter any errors in the below cell, please try to restart the kernel (Kernel -> Restart & Run All)
</div>

In [ ]:
req_packages = [    
                    {"sciunit"                  : {"min_version": "0.2.1",  "install_version": "0.2.1"}},
                    {"bluepyopt"                : {"min_version": "1.6.22", "install_version": "1.6.42"}},
                    {"basalunit"                : {"min_version": "1.0.4",  "install_version": "1.0.4"}},
                    {"hbp_validation_framework" : {"min_version": "0.5.19", "install_version": "0.5.19"}},
                    {"numpy"                    : {"min_version": "1.16.2", "install_version": "1.16.2"}},
                    {"Jinja2"                   : {"min_version": "2.10.3", "install_version": "2.10.3"}},
                    {"tornado"                  : {"min_version": "4.5.3",  "install_version": "4.5.3"}}
                ]

def install_req_packages():
    # currently handles installations via PyPI and GitHub
    for pkg in req_packages:        
        for pkg_name, pkg_vinfo in pkg.items():
            print("Checking for package: {}".format(pkg_name))        
            try:
                pkg_resources.get_distribution(pkg_name)        
                current_version = parse_version(pkg_resources.get_distribution(pkg_name).version)
                print("\t{}: current version = {}".format(pkg_name, current_version))
                if not pkg_vinfo["min_version"] or current_version < parse_version(pkg_vinfo["min_version"]) or current_version > parse_version(pkg_vinfo["install_version"]):                                                
                        print("\tInstalling another version of {}.".format(pkg_name))
                        raise
            except:            
                if "github.com" in pkg_vinfo["install_version"]:
                    os.system("pip install --quiet --no-cache-dir --force-reinstall git+{}".format(pkg_vinfo["install_version"]))
                else:
                    os.system("pip install --quiet --no-cache-dir --force-reinstall {}=={}".format(pkg_name, pkg_vinfo["install_version"]))                                
                print("\t{}: installed version = {}".format(pkg_name, pkg_vinfo["install_version"]))                

install_req_packages()                 

In [ ]:
import ast
import json
import tarfile
import urllib
import sciunit
import pandas as pd
from pandas.io.json import json_normalize
import bluepyopt.ephys as ephys
from basalunit.utils import CellModel
from hbp_validation_framework import utils, TestLibrary, ModelCatalog

## 2. Check if Model Catalog and Validation Framework Apps Exist in Collab
If the notebook is run inside a Collab, we check if an instance of the Model Catalog and Validation Framework apps exist in the current Collab. If not, we add an instance of each (this will be reflected in the Collab's navigation panel, possibly on reloading the page).

NOTE: **HBP_USERNAME** is an optional parameter when the notebook is being run inside the Collaboratory. The notebook can automatically identify your username in this scenario. This parameter needs to be specified if a user wishes to download the notebook and run it locally. Another potential (less likely) reason for specifying this (even within the Collaboratory) is in dealing with access permissions (wanting to run the test with different credentials).

NOTE: Even if this notebook is not run inside a Collab, the following cell needs to be executed. It will identify if environment and manage accordingly. When not run inside a Collab, it will simply setup parameters required for the test, and not attempt to create new apps.

In [ ]:
# your HBP username; not essential if running inside the Collaboratory
HBP_USERNAME = ""
testLibrary = TestLibrary(username=HBP_USERNAME)
modelCatalog = ModelCatalog.from_existing(testLibrary)

try:
    collab_path = get_collab_storage_path()
    collab_id = collab_path[1:] # this might fail for very old Collabs which use name instead of Collab ID
except:
    # not run inside Collaboratory
    print("\nPlease enter a Collab ID where you wish to store the results:")
    print("E.g.: 8123")
    print("Note: you should be a member of this Collab!")
    collab_id = input()
    if not isinstance(collab_id, int):
        raise ValueError("Possibly invalid Collab ID: {}. Numeric input expected!".format(collab_id))    

# check if apps exist; if not then create them
MCapp_navID = modelCatalog.exists_in_collab_else_create(collab_id)
modelCatalog.set_app_config(collab_id=collab_id, app_id=MCapp_navID, only_if_new="True")
VFapp_navID = testLibrary.exists_in_collab_else_create(collab_id)
testLibrary.set_app_config(collab_id=collab_id, app_id=VFapp_navID, only_if_new="True")

## 3. Model Selection: Specifying model from ModelCatalog
The user is given an option to choose from existing compatible models.

In [ ]:
m1 = modelCatalog.get_model(model_id="c27e711e-75e3-498f-bc64-6242ea5d8772") # d9c2404f-427f-4205-8aa3-6989131ce96e
m2 = modelCatalog.get_model(model_id="6deb0c49-e762-4b60-8206-a46322bca3db") # 36c1dcb2-779c-4b44-b842-ecb255a3fbbe
m3 = modelCatalog.get_model(model_id="0b2fda97-2403-482b-90d1-738d1b6d013c") # c1c513e3-8ada-4091-ba3a-22537ccf05eb
list_of_models = [m1, m2, m3]
df = pd.DataFrame.from_dict(json_normalize(list_of_models), orient='columns')
df = df.reindex(columns=['name', 'id', 'author', 'brain_region', 'species', 'cell_type', 'model_scope', 'abstraction_level', 'description'])
df.index += 1 
print("Available models are listed below:")
df.replace('\n','', regex=True)

In [ ]:
print("Enter the # of required model: ")
choice = int(input())
if choice <= len(list_of_models) and choice > 0:   
    model_id = list_of_models[choice-1]["id"]
    model_name = list_of_models[choice-1]["name"]
else:
    raise ValueError("Invalid entry for model choice!")    

### 3.1 Gather Additional Info
Currently the test is applicable for three cell types: _D1-type MSN_, _D2-type MSN_ and _FS Interneurons_ <br />
The usecase tries to identify the cell type from the model name. If this isn't possible, the user is prompted to specify the cell type.

Experimental recordings might specify a value for their junctional potential. For the observation data used here, the same was recorded as 9.5 mV and is therefore used as the default value. The user is prompted to change this if needed.

In [ ]:
# will try to determine cell type from model name; if not then ask user
# current options: msn_d1, msn_d2, fs
if "msn_d1" in model_name:
    cell_type = "msn_d1" 
elif "msn_d2" in model_name:
    cell_type = "msn_d2"
elif "-fs-" in model_name:
    cell_type = "fs"    
else:
    print("\nPlease enter the cell_type: ")
    options = ["msn_d1", "msn_d2", "fs"]
    for i, each in enumerate(options,start=1):
        print("\t{}. {}".format(i,each))
    print("Enter the # of cell_type: ")
    choice = int(input())
    cell_type = options[choice-1]
print("Cell Type = {}".format(cell_type))
    
print("\nEnter a value for junction potential of experimental recordings (in mV):")
print("Note: leave blank to use default value (i.e. 9.5); enter numeric value, no units")
jp_value = input()
if jp_value == "":
    junction_potential = 9.5 # mV
else:
    junction_potential = float(jp_value)
print("Junction Potential = {} mV".format(junction_potential))

### 3.2 Selection of desired model instance
The model has several model instances corresponding to the Hall of Fame (HoF) parameter sets found in the model's "hall_of_fame.json" file. These are listed below. The user needs to select one of these model instances.

The model morphology file (.swc) that was found within the model directory, and used to construct the model, is indicated.

In [ ]:
model_instances = modelCatalog.list_model_instances(model_id=model_id)
df = pd.DataFrame.from_dict(json_normalize(model_instances), orient='columns')
df = df.reindex(columns=['version', 'id', 'parameters'])
df.index += 1
pd.set_option('display.max_colwidth', -1)
df.replace('\n','', regex=True)

The user is asked to specify the model instances that they are interested in validating. The valid range of values are indicated. The user can provide input as follows: <br />
 * All parameters, specify: `all` <br />
 * Single parameter set, specify number, e.g.: `1` <br />
 * Multiple parameter sets, specify numbers as a list, e.g.: `[1,4,5,8]` <br />
 
**Note:** validation of each parameter set takes around 20 mins; for testing specify a single parameter set.

In [ ]:
print("Enter the model instance(s) to be validated: 1 - {}".format(len(model_instances)))
print("Example inputs: 1, 4, [1,4,5,8], all")
instances_entry = input().lower()
if instances_entry == "all":
    instances_list = range(1, len(model_instances)+1)
else:    
    if isinstance(eval(instances_entry), list):
        instances_list = eval(instances_entry)        
    elif isinstance(eval(instances_entry), int):
        instances_list = [eval(instances_entry)]
    else:
        raise ValueError("Invalid entry for parameter set!")  
       
valid_instances_list = []
for i in instances_list:
    if i > 0 and i <= len(model_instances):
        valid_instances_list.append(i)
    else:
        print("Invalid entry: {}. Excluded.".format(i))

In [ ]:
try:
    # Note: For all models used in this UseCase, all the model instances have same source code
    # They differ only in the parameter values assigned
    file_path = model_instances[0]["source"]
    file_tmp = urllib.request.urlretrieve(file_path, filename=os.path.basename(file_path))[0]
    file_path = os.path.join(os.getcwd(),os.path.basename(file_path))
except:
    raise IOError("Model url = {} is invalid!".format(file_path))    
    
cell_model = CellModel(model_path=file_path, cell_type=cell_type, model_name=model_name)
cell_model.model_uuid = model_id

## 4. Instantiating the model(s); Running Model Validation Test


**Note:** It takes roughly 20 minutes for each model instance. At the end of the test, the user is provided with a textual summary of the _score_ and the path to related output files generated by the test. These and other details can be viewed in the  _Validation Framework_ app (see Collab's Navigation panel; select `Validation Framework`).

In [ ]:
result_uuids = []

for index in valid_instances_list:
    cell_model.model_version = model_instances[index-1]["version"]
    cell_model.params = ast.literal_eval(model_instances[index-1]["parameters"])
    
    print("NOTE: The simulation(s) may take some time to complete (typically 10-15 minutes). Please wait.")
    result_id, score = utils.run_test(username=HBP_USERNAME, model=cell_model, test_alias="basalg_" + cell_type, test_version="1.0", storage_collab_id=collab_id, register_result=True, client_obj=testLibrary, cell_type=cell_type, junction_potential=junction_potential, use_cache=True)
    result_uuids.append(result_id)    

print("The result(s) can be viewed in the HBP Validation Framework app. Direct link(s):")
for result_uuid in result_uuids:
    print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=result.{}".format(str(collab_id),str(VFapp_navID), result_uuid))

### 4.1 Score Summary

In [ ]:
if len(result_uuids) > 0:
    df, excluded_results = utils.generate_score_matrix(result_list=result_uuids, collab_id=collab_id, client_obj=modelCatalog)        
    from IPython.core.display import HTML
    HTML("<style>.rendered_html th {max-width: 120px;}</style>")
    display(df)

### 4.2 Generate Report
The validation framework can generate an HTML report for all the successfully completed tests. The user is prompted whether such a report should be generated for the current validation results. If asked to generate, the location to the generated HTML is indicated.

In [ ]:
report_path = None
if len(result_uuids) > 0:
    print("\nDo you wish to generate an HTML report of the executed tests?")
    print("Enter: y/n")
    choice = input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        report_path, valid_uuids = utils.generate_HTML_report(result_list=result_uuids, collab_id=collab_id, client_obj=modelCatalog)    

### 4.3 View Report Inside Jupyter Notebook
The HTML report created in the above cell is displayed within the Jupyter notebook.

In [ ]:
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width="100%", height=1000))